## Make Datasets
하다하다 못해먹겠어서 내가 직접 만드는 데이트셋

In [1]:
!pip install pandas
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import json
import os
from datasets import Dataset, Audio

# import
import os
import json
import torch
import torchaudio
from torch.utils.data import DataLoader
import torch.nn as nn

from transformers import Wav2Vec2ForCTC, AutoProcessor,Wav2Vec2CTCTokenizer,Wav2Vec2Processor

/home/haram/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-18 20:09:55.552662: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-18 20:09:55.605315: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 20:09:55.605362: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 20:09:55.605426: E 

In [3]:
# JSON 파일 경로
json_file_path = "./data/exist_test/rami_mapping.json"

# JSON 파일 읽기
with open(json_file_path, "r", encoding="utf-8") as json_file:
    mapping_Data = json.load(json_file)

In [4]:
df = pd.DataFrame(mapping_Data)


# change name
df = df.rename(columns={"path": "audio"})

# dataset = Dataset.from_pandas(df).cast_column("audio", Audio(sampling_rate=16_000))

# print(dataset)

In [5]:
from transformers import AutoProcessor

model_name = "facebook/wav2vec2-base-960h"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)
processor = Wav2Vec2Processor.from_pretrained(model_name)


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [6]:
def prepare_dataset(batch):
    batch["input_values"] = []
    batch["input_length"] = []
    
    for audio in batch["audio"]:
        input_values = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
        input_length = len(input_values)
        
        batch["input_values"].append(input_values)
        batch["input_length"].append(input_length)
    

    print("tra",batch["transcription"])
    batch["labels"] = processor(text=batch["transcription"]).input_id
    return batch

dataset = Dataset.from_pandas(df)
dataset.set_transform(prepare_dataset)

dataset

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 200
})

In [7]:
# 각 예제에 대해 평가 수행
for example in dataset:
    # 오디오 파일 경로
    audio_path = example["audio"]
    # 오디오 파일 로드
    speech, _ = torchaudio.load(audio_path)
    # 모델 입력값으로 변환
    inputs = processor(input_data=speech, sampling_rate=16000, return_tensors="pt", padding=True)
    # 모델 추론
    with torch.no_grad():
        logits = model(**inputs).logits
    # CTC 디코딩을 사용하여 텍스트로 변환
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    # 정답과 비교하여 정확도 업데이트
    if transcription[0] == example["transcription"]:
        correct_predictions += 1

# 정확도 출력
accuracy = correct_predictions / total_samples
print("Accuracy:", accuracy)

NameError: name 'common_voice' is not defined